## Importación de librerías

In [558]:
import pandas as pd

## Limpieza de datos para reducción de features

**Importación de CSV**

In [559]:
df = pd.read_csv("../Input/athlete_events.csv")  

In [560]:
df.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN


**Si queremos agrupar por país (lo más requerido, probablemente), hay que limpiar su información**

In [561]:
df["NOC"].value_counts()

USA    18853
FRA    12758
GBR    12256
ITA    10715
GER     9830
       ...  
YMD        5
SSD        3
UNK        2
NBO        2
NFL        1
Name: NOC, Length: 230, dtype: int64

**Importamos el dataset de optimización de países**

In [562]:
df2 = pd.read_csv("../Input/noc_regions.csv")  

df2.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [563]:
df2 = df2.drop(columns=["notes"])

In [564]:
df2.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


**Fusionamos los datasets**

In [565]:
df3 = df.merge(df2, on='NOC')

In [566]:
df3.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China


**Continuamos la limpieza tradicional...**

In [567]:
df3 = df3.loc[df3["Season"] == "Summer"].drop(columns="Season")

In [568]:
df3 = pd.get_dummies(df3,columns=["Sex"])

In [569]:
df3 = df3.drop(columns=["ID","Games","Event","Team","NOC","Height","Weight"])

In [570]:
df3 = df3.rename(columns={"City": "Host City", "region": "Region","Name":"Athlete"})

In [571]:
df3.head()

,Athlete,Age,Year,Host City,Sport,Medal,Region,Sex_F,Sex_M
0,A Dijiang,24.0,1992,Barcelona,Basketball,NaN,China,0,1
1,A Lamusi,23.0,2012,London,Judo,NaN,China,0,1
2,Abudoureheman,22.0,2000,Sydney,Boxing,NaN,China,0,1
3,Ai Linuer,25.0,2004,Athina,Wrestling,NaN,China,0,1
4,Ai Yanhan,14.0,2016,Rio de Janeiro,Swimming,NaN,China,1,0


**Obtenemos los países anfitriones a partir de las ciudades**

In [572]:
df4 = pd.read_csv("../Input/host_cities.csv")  

In [573]:
df4.head()

,Host City,Host Country
0,Barcelona,ESP
1,London,GBR
2,Antwerpen,BEL
3,Paris,FRA
4,Calgary,CAN


In [574]:
df5 = df3.merge(df4, on='Host City').drop(columns="Host City")

In [575]:
df5.head()

,Athlete,Age,Year,Sport,Medal,Region,Sex_F,Sex_M,Host Country
0,A Dijiang,24.0,1992,Basketball,NaN,China,0,1,ESP
1,Bai Chongguang,21.0,1992,Boxing,NaN,China,0,1,ESP
2,Bai Mei,17.0,1992,Rhythmic Gymnastics,NaN,China,1,0,ESP
3,Bi Zhong,23.0,1992,Athletics,NaN,China,0,1,ESP
4,Cai Yanshu,28.0,1992,Weightlifting,NaN,China,0,1,ESP


**Reordenamos las columnas del dataset para mejor comprensión**

In [576]:
df5 = df5[["Host Country", "Athlete", "Age", "Year", "Sport", "Region", "Sex_F", "Sex_M", "Medal"]]

df5.head()

,Host Country,Athlete,Age,Year,Sport,Region,Sex_F,Sex_M,Medal
0,ESP,A Dijiang,24.0,1992,Basketball,China,0,1,NaN
1,ESP,Bai Chongguang,21.0,1992,Boxing,China,0,1,NaN
2,ESP,Bai Mei,17.0,1992,Rhythmic Gymnastics,China,1,0,NaN
3,ESP,Bi Zhong,23.0,1992,Athletics,China,0,1,NaN
4,ESP,Cai Yanshu,28.0,1992,Weightlifting,China,0,1,NaN


**Conversión de valores categóricos a numéricos**

In [577]:
df5.dtypes

Host Country     object
Athlete          object
Age             float64
Year              int64
Sport            object
Region           object
Sex_F             uint8
Sex_M             uint8
Medal            object
dtype: object

In [578]:
df5 = pd.get_dummies(df5,columns=["Medal"])

In [579]:
df5 = pd.DataFrame(df5.groupby(['Host Country','Year','Region']).agg({'Athlete':"nunique", 'Sport':"nunique",'Medal_Gold':"sum",'Medal_Silver':"sum", 'Medal_Bronze':"sum"}))

df5.reset_index().tail()

,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze
2788,USA,1996,"Virgin Islands, British",7,2,0,0,0
2789,USA,1996,"Virgin Islands, US",12,5,0,0,0
2790,USA,1996,Yemen,4,2,0,0,0
2791,USA,1996,Zambia,8,2,0,1,0
2792,USA,1996,Zimbabwe,13,6,0,0,0


**Continuamos la limpieza tradicional**

In [580]:
df5['Medals'] = df5.loc[:, 'Medal_Gold':'Medal_Bronze'].sum(1)

df5 = df5.reset_index()

df5.head()

,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze,Medals
0,AUS,1956,Afghanistan,12,1,0,0,0,0
1,AUS,1956,Argentina,28,8,0,1,1,2
2,AUS,1956,Australia,294,18,28,13,26,67
3,AUS,1956,Austria,29,11,0,0,4,4
4,AUS,1956,Bahamas,4,2,0,0,2,2


In [581]:
df5 = df5.loc[df5["Year"]>=1990].sort_values(by=["Year"]).reset_index(drop=True)

print(df5.shape)

df5.head()

(1365, 9)


,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze,Medals
0,ESP,1992,Kuwait,32,7,0,0,0,0
1,ESP,1992,Niger,3,1,0,0,0,0
2,ESP,1992,Nigeria,55,8,0,7,4,11
3,ESP,1992,North Korea,64,12,4,0,6,10
4,ESP,1992,Norway,83,17,2,20,1,23


**Exportación de CSV**

In [582]:
df5.to_csv("Csv4.csv")

In [583]:
df5 = df5.loc[df5["Year"]==2016].sort_values(by=["Year"]).reset_index(drop=True)

In [584]:
df6 = df5[["Host Country", "Year", "Region", "Athlete", "Sport","Medals"]]

df6

,Host Country,Year,Region,Athlete,Sport,Medals
0,BRA,2016,Oman,4,2,0
1,BRA,2016,Albania,6,3,0
2,BRA,2016,Algeria,64,13,2
3,BRA,2016,Andorra,4,4,0
4,BRA,2016,Angola,26,7,0
...,...,...,...,...,...,...
198,BRA,2016,Somalia,2,1,0
199,BRA,2016,Turkey,100,21,8
200,BRA,2016,Turkmenistan,9,5,0
201,BRA,2016,Swaziland,2,1,0
